# Setup

In [1]:
!pip install -q tf-models-official==2.3.0
!pip install transformers

In [2]:
import os
import json

import re

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

from official.modeling import tf_utils
from official import nlp
from official.nlp import bert

# Load the required submodules
import official.nlp.optimization
import official.nlp.bert.bert_models
import official.nlp.bert.configs
import official.nlp.bert.run_classifier
import official.nlp.bert.tokenization
import official.nlp.data.classifier_data_lib
import official.nlp.modeling.losses
import official.nlp.modeling.models
import official.nlp.modeling.networks

This directory contains the configuration, vocabulary, and a pre-trained checkpoint used in this tutorial:

In [3]:
gs_folder_bert = "gs://cloud-tpu-checkpoints/bert/keras_bert/uncased_L-12_H-768_A-12"
tf.io.gfile.listdir(gs_folder_bert)

['bert_config.json',
 'bert_model.ckpt.data-00000-of-00001',
 'bert_model.ckpt.index',
 'vocab.txt']

In [4]:
hub_url_bert = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2"

# Dataset

In [5]:
# import sys
# !git clone "https://github.com/giuliofortini/NLP_SQuAD_Project"
# sys.path.append("NLP_SQuAD_Project/")

from google.colab import files
try:
  with open('training_set.json') as f:
    json_data = json.load(f)
except:
  files.upload()

In [6]:
data = pd.json_normalize(json_data['data'])  
#data = pd.json_normalize(json_data['data'], ['paragraphs'])
data

,title,paragraphs
0,University_of_Notre_Dame,"[{'context': 'Architecturally, the school has ..."
1,Beyoncé,[{'context': 'Beyoncé Giselle Knowles-Carter (...
2,Montana,[{'context': 'Montana i/mɒnˈtænə/ is a state i...
3,Genocide,"[{'context': 'The phrase ""in whole or in part""..."
4,Antibiotics,[{'context': 'The emergence of resistance of b...
...,...,...
437,Police,[{'context': 'A police force is a constituted ...
438,"Punjab,_Pakistan","[{'context': 'Punjab (Urdu, Punjabi: پنجاب, pa..."
439,Infection,[{'context': ' Among the vast varieties of mic...
440,Hunting,[{'context': 'Hunting is the practice of killi...


In [7]:
temp = []
title_dict = {}
contexts = []

for i, row in data.iterrows():
  for context in row['paragraphs']:
    contexts.append(context['context'])
    for qa in context['qas']:
      question_id = qa['id']
      question = qa['question']
      for answer in qa['answers']:
        answer_text = answer['text']
        answer_start = answer['answer_start']
        answer_end = answer_start+len(answer_text)
        temp.append([question_id, question, answer_text, answer_start, answer_end, i, len(contexts)-1])


context_dict = dict(enumerate(contexts))

df = pd.DataFrame(temp, columns=['question_id', 'question_text', 'answer_text', 'answer_start', 'answer_end', 'title_id', 'context_id'])
#df['context_id'] = context_df.index
context_dict[0]

'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.'

In [8]:
df

,question_id,question_text,answer_text,answer_start,answer_end,title_id,context_id
0,5733be284776f41900661182,To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous,515,541,0,0
1,5733be284776f4190066117f,What is in front of the Notre Dame Main Building?,a copper statue of Christ,188,213,0,0
2,5733be284776f41900661180,The Basilica of the Sacred heart at Notre Dame...,the Main Building,279,296,0,0
3,5733be284776f41900661181,What is the Grotto at Notre Dame?,a Marian place of prayer and reflection,381,420,0,0
4,5733be284776f4190066117e,What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary,92,126,0,0
...,...,...,...,...,...,...,...
87594,5735d259012e2f140011a09d,In what US state did Kathmandu first establish...,Oregon,229,235,441,18895
87595,5735d259012e2f140011a09e,What was Yangon previously known as?,Rangoon,414,421,441,18895
87596,5735d259012e2f140011a09f,With what Belorussian city does Kathmandu have...,Minsk,476,481,441,18895
87597,5735d259012e2f140011a0a0,In what year did Kathmandu create its initial ...,1975,199,203,441,18895


In [9]:
import random
def print_squad_sample(train_data, line_length=120, separator_length=150):
  sample = train_data.sample(frac=1).head(1)
  context = context_dict[sample['context_id'].item()]
  print('='*separator_length)
  print('CONTEXT: ')
  print('='*separator_length)
  lines = [''.join(context[idx:idx+line_length]) for idx in range(0, len(context), line_length)]
  for l in lines:
      print(l)
  #print(context)
  print('='*separator_length)
  questions = train_data[train_data['context_id'] == sample['context_id'].item()]
  print('QUESTION:', ' '*(3*separator_length//4), 'ANSWER:')
  for idx, row in questions.iterrows():
    question = row.question_text
    answer = row.answer_text
    print(question, ' '*(3*separator_length//4-len(question)+9), answer)

print_squad_sample(df)

CONTEXT: 
In 1409, a hospital was founded and placed under the patronage of Santa María de los Inocentes; to this was attached a c
onfraternity devoted to recovering the bodies of the unfriended dead in the city and within a radius of three miles (4.8
 km) around it. At the end of the 15th century this confraternity separated from the hospital, and continued its work un
der the name of "Cofradia para el ámparo de los desamparados". King Philip IV of Spain and the Duke of Arcos suggested t
he building of the new chapel, and in 1647 the Viceroy, Conde de Oropesa, who had been preserved from the bubonic plague
, insisted on carrying out their project. The Blessed Virgin was proclaimed patroness of the city under the title of Vir
gen de los desamparados (Virgin of the Forsaken), and Archbishop Pedro de Urbina, on 31 June 1652, laid the cornerstone 
of the new chapel of this name. The archiepiscopal palace, a grain market in the time of the Moors, is simple in design,
 with an inside cloist

# Encoding

#Tokenizer

In [100]:
# Set up tokenizer to generate Tensorflow dataset
tokenizer = bert.tokenization.FullTokenizer(
    vocab_file=os.path.join(gs_folder_bert, "vocab.txt"),
     do_lower_case=True)

print("Vocab size:", len(tokenizer.vocab))

Vocab size: 30522


In [101]:
# Tokenize our sentence with the BERT tokenizer.
tokens = tokenizer.tokenize(df['question_text'][0])

# Print out the tokens.
print(tokens)

['to', 'whom', 'did', 'the', 'virgin', 'mary', 'allegedly', 'appear', 'in', '1858', 'in', 'lou', '##rdes', 'france', '?']


In [102]:
tokenizer.convert_tokens_to_ids(['[CLS]', '[SEP]'])

[101, 102]

In [103]:
# Split the sentence into tokens.
tokenized_text = tokenizer.tokenize(df['question_text'][0])

# Print out the tokens.
print(tokenized_text)

# Map the token strings to their vocabulary indeces.
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

# Display the words with their indeces.
for tup in zip(tokenized_text, indexed_tokens):
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

['to', 'whom', 'did', 'the', 'virgin', 'mary', 'allegedly', 'appear', 'in', '1858', 'in', 'lou', '##rdes', 'france', '?']
to            2,000
whom          3,183
did           2,106
the           1,996
virgin        6,261
mary          2,984
allegedly     9,382
appear        3,711
in            1,999
1858          8,517
in            1,999
lou          10,223
##rdes       26,371
france        2,605
?             1,029


In [104]:
from keras.preprocessing.sequence import pad_sequences
import re

# convert tokens to ids
def tokens_to_int(column):
  return tokenizer.convert_tokens_to_ids(column)

# convert ids to tokens
def int_to_token(id):
  return tokenizer.convert_ids_to_tokens(id)


# tokenize non-context column
def tokenize_text(text):
  return ['[CLS]'] + tokenizer.tokenize(text) + ['[SEP]']

# tokenize context column
def tokenize_context(idx):
  context_dict[idx] = context_dict[idx].replace('."', '".')
  sentences = re.split('(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', context_dict[idx].strip())
  tokenized_sentence = []
  for sentence in sentences:
    tok = tokenizer.tokenize(sentence) + ["[SEP]"]
    tokenized_sentence += tok
  return tokenized_sentence #tokenizer.tokenize(re.sub('(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', ' [SEP] ', context_dict[idx].strip())) + ['[SEP]']

# create column mask
def get_mask(tokenized_column, max_len):
  mask = []
  for row in tokenized_column:
    mask.append(tf.concat((tf.zeros(max_len-len(row)), tf.ones_like(row, dtype=tf.float32)), axis=0))
  return mask

# return tokenized df with input masks
def process_df(df):
  
  # initialize structures
  columns = ['question_text', 'context'] #, 'answer_text']
  processed_df = pd.DataFrame(columns=columns)
  input_mask = []

  for column in columns:
    print("processing column: ", column)
    
    if column == 'question_text':
      # tokenize column
      tokenized_column = df[column].apply(tokenize_text).apply(tokens_to_int)
      processed_df[column] = pad_sequences(tokenized_column, padding="post").tolist()
      # create input mask
      max_len = len(processed_df[column][0])
      input_mask = get_mask(tokenized_column, max_len)
      processed_df[column+'_mask'] = input_mask
    # elif column == 'answer_text':
    #   # answer_text len must be equale to len(padded question_text) + len(padded context)
    #   max_len = len(processed_df['question_text'][0]) + len(processed_df['context'][0])
    #   # tokenize column
    #   tokenized_column = df[column].apply(tokenize_text).apply(tokens_to_int)
    #   processed_df[column] = pad_sequences(tokenized_column, max_len).tolist()
    #   # create input mask
    #   input_mask = get_mask(tokenized_column, max_len)
    #   processed_df[column+'_mask'] = input_mask
    else:
      # tokenize column
      tokenized_column = df['context_id'].apply(tokenize_context).apply(tokens_to_int)
      processed_df[column] = pad_sequences(tokenized_column, padding="post").tolist()
      # create input mask
      max_len = len(processed_df[column][0])
      input_mask = get_mask(tokenized_column, max_len)
      processed_df['context_mask'] = input_mask
  
  # add useful columns
  processed_df['question_id'] = df['question_id']
  processed_df['answer_start'] = df['answer_start']
  processed_df['answer_end'] = df['answer_end']
  
  return processed_df



In [105]:
context_bkp = context_dict.copy()
context_bkp[0]

'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.'

In [106]:
# CONTEXT_LIMIT = 2300
# mx = 0
# for id in context_dict:
#   if len(context_dict[id]) > mx: mx = len(context_dict[id])
#   context_dict[id] = context_dict[id][:CONTEXT_LIMIT]

# print("max: ", mx)

In [107]:
processed_df = process_df(df)
processed_df

processing column:  question_text
processing column:  context


,question_text,context,question_text_mask,context_mask,question_id,answer_start,answer_end
0,"[101, 2000, 3183, 2106, 1996, 6261, 2984, 9382...","[6549, 2135, 1010, 1996, 2082, 2038, 1037, 323...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...",5733be284776f41900661182,515,541
1,"[101, 2054, 2003, 1999, 2392, 1997, 1996, 1028...","[6549, 2135, 1010, 1996, 2082, 2038, 1037, 323...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...",5733be284776f4190066117f,188,213
2,"[101, 1996, 13546, 1997, 1996, 6730, 2540, 201...","[6549, 2135, 1010, 1996, 2082, 2038, 1037, 323...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...",5733be284776f41900661180,279,296
3,"[101, 2054, 2003, 1996, 24665, 23052, 2012, 10...","[6549, 2135, 1010, 1996, 2082, 2038, 1037, 323...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...",5733be284776f41900661181,381,420
4,"[101, 2054, 7719, 2006, 2327, 1997, 1996, 2364...","[6549, 2135, 1010, 1996, 2082, 2038, 1037, 323...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...",5733be284776f4190066117e,92,126
...,...,...,...,...,...,...,...
87594,"[101, 1999, 2054, 2149, 2110, 2106, 28045, 203...","[28045, 4956, 2103, 1006, 2463, 2278, 1007, 10...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...",5735d259012e2f140011a09d,229,235
87595,"[101, 2054, 2001, 8675, 2239, 3130, 2124, 2004...","[28045, 4956, 2103, 1006, 2463, 2278, 1007, 10...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...",5735d259012e2f140011a09e,414,421
87596,"[101, 2007, 2054, 19337, 21694, 17043, 2103, 2...","[28045, 4956, 2103, 1006, 2463, 2278, 1007, 10...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...",5735d259012e2f140011a09f,476,481
87597,"[101, 1999, 2054, 2095, 2106, 28045, 3443, 204...","[28045, 4956, 2103, 1006, 2463, 2278, 1007, 10...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...","(tf.Tensor(0.0, shape=(), dtype=float32), tf.T...",5735d259012e2f140011a0a0,199,203


In [108]:
print(len(processed_df['context'][0]))

880


In [109]:
question_len = len(processed_df['question_text_mask'][0])
context_len = len(processed_df['context_mask'][0])
input_len = question_len + context_len
#answer_len = len(processed_df['answer_text_mask'][0])

print("question_text_mask len: ", question_len)
print("context_mask len: ", context_len)
print("input len: ", input_len)
#print("answer_text_mask len: ", answer_len)

question_text_mask len:  63
context_mask len:  880
input len:  943


In [110]:
def preprocess_input(processed_df):
  
  # initialize structures
  input = []  
  input_mask = []  
  input_type_ids = []

  print("\nprocessing input, input_type_ids and input_mask...\n")
  for i, row in processed_df.iterrows():

    if i % 10000 == 0:
      print(i)
    
    # create input tensor by concatenation of question_text and context
    input.append(tf.convert_to_tensor(row['question_text'] + 
                                      row['context'] + 
                                      [row['answer_start']] + 
                                      [row['answer_end']] ))

    # create type_ids tensor (needed for model input)
    type_question = tf.zeros_like(row['question_text'])
    type_context = tf.zeros_like(row['context'])
    type_answer_idxs = tf.ones(2, dtype=tf.int32)
    input_type_ids.append(tf.concat([type_question, type_context, type_answer_idxs], axis=-1))

    # create input attention mask based on padding
    input_mask.append(tf.concat((row['question_text_mask'], row['context_mask'], [1], [1]), axis=-1))

  print("\ncreating dictionary...", end="")
  # save input data as dictionary
  inputs = {
    'input_ids': tf.convert_to_tensor(input),
    'input_mask': tf.convert_to_tensor(input_mask),
    'input_type_ids': tf.convert_to_tensor(input_type_ids)
    }
    
  print("done!\n")
  return inputs

model_input = preprocess_input(processed_df)
#model_labels = tf.convert_to_tensor(list(processed_df['answer_text']))

print('input_ids:', model_input['input_ids'])
print('input_mask: ', model_input['input_mask'])
print('input_type_ids:', model_input['input_type_ids'])
# print('labels: ', model_labels)


processing input, input_type_ids and input_mask...

0
10000
20000
30000
40000
50000
60000
70000
80000

creating dictionary...done!

input_ids: tf.Tensor(
[[  101  2000  3183 ...     0   515   541]
 [  101  2054  2003 ...     0   188   213]
 [  101  1996 13546 ...     0   279   296]
 ...
 [  101  2007  2054 ...     0   476   481]
 [  101  1999  2054 ...     0   199   203]
 [  101  2054  2003 ...     0     0    27]], shape=(87599, 945), dtype=int32)
input_mask:  tf.Tensor(
[[0. 0. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 1. 1. 1.]
 ...
 [0. 0. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 1. 1. 1.]], shape=(87599, 945), dtype=float32)
input_type_ids: tf.Tensor(
[[0 0 0 ... 0 1 1]
 [0 0 0 ... 0 1 1]
 [0 0 0 ... 0 1 1]
 ...
 [0 0 0 ... 0 1 1]
 [0 0 0 ... 0 1 1]
 [0 0 0 ... 0 1 1]], shape=(87599, 945), dtype=int32)


In [111]:
# Set up epochs and steps
epochs = 3
batch_size = 32
eval_batch_size = 32

train_data_size = len(model_input['input_ids'])
steps_per_epoch = int(train_data_size / batch_size)
num_train_steps = steps_per_epoch * epochs
warmup_steps = int(epochs * train_data_size * 0.1 / batch_size)

# # creates an optimizer with learning rate schedule
# optimizer = nlp.optimization.create_optimizer(
#     2e-5, num_train_steps=num_train_steps, num_warmup_steps=warmup_steps)
# metrics = [tf.keras.metrics.SparseCategoricalAccuracy('accuracy', dtype=tf.float32)]
# loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [112]:
from transformers import TFBertModel
bert_pretrained = TFBertModel.from_pretrained('bert-large-uncased')

Some layers from the model checkpoint at bert-large-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-large-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [113]:
tf.keras.utils.plot_model(bert_pretrained, show_shapes=True, dpi=48)

In [116]:
# compute the max distance between consequent 'id' occurrences
def max_distance(arr, id):
  positions = np.where(np.array(arr) == id)[0]
  distances = [positions[i+1] - positions[i] - 1 for i in range(len(positions)-1)]
  return max(distances)

# find the too long sentence (having more than 512 tokens between two SEP token)
count = 0
sentences_lengths = []
for i, row in enumerate(model_input["input_ids"].numpy()):
  dist = max_distance(row, 102)
  if dist >= 512: # sentences longer than 512 are rejected by BERT
    sentences_lengths.append((dist, i))
    count += 1
  # print(row)

print(f"Found {count} sentences longer than 512 tokens.")

if len(sentences_lengths) != 0:
  max_length, longest_id = max(sentences_lengths)
  toolong = context_dict[df["context_id"][longest_id]]
  tokenized_toolong = model_input["input_ids"][longest_id].numpy()
  tokenized_toolong = [t for t in tokenized_toolong]
  tokenized_toolong = ' '.join(int_to_token(tokenized_toolong))
  print(f"Longest sentences's length: {max_length}")
  print(f"\nLongest sentence (id {longest_id}): \n  {toolong}")
  print(f"\nTokenized: \n  {tokenized_toolong}\n\n")

  for sent in tokenized_toolong.split("[SEP]"):
    print(f"> {len(sent.strip().split(' ')) }\ttokens:   {sent.strip()}")

Found 318 sentences longer than 512 tokens.
Longest sentences's length: 852

Longest sentence (id 18797): 
  Slavic studies began as an almost exclusively linguistic and philological enterprise. As early as 1833, Slavic languages were recognized as Indo-European.

Tokenized: 
  [CLS] when were slavic languages recognized as indo - european ? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] slavic studies began as an almost exclusively linguistic and phil ##ological enterprise . [SEP] as early as 1833 , slavic languages were recognized as indo - european . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

In [115]:
for i in range(0, steps_per_epoch):
  input = {'input_ids' : model_input['input_ids'][i:i+batch_size],
           'input_mask' : model_input['input_mask'][i:i+batch_size],
           'input_type_ids' : model_input['input_type_ids'][i:i+batch_size]
           }
  print(input['input_ids'])
  bert_pretrained(input, training = True)
  break

# bert_pretrained(model_input, training = True)
# for i in range(0, train_data_size):
#   input = {'input_ids' : model_input['input_ids'][i],
#            'input_mask' : model_input['input_mask'][i],
#            'input_type_ids' : model_input['input_type_ids'][i]
#            }
#   bert_pretrained( input_ids = model_input['input_ids'][i], attention_mask = model_input['input_mask'][i], token_type_ids = model_input['input_type_ids'][i] , training = True)

tf.Tensor(
[[  101  2000  3183 ...     0   515   541]
 [  101  2054  2003 ...     0   188   213]
 [  101  1996 13546 ...     0   279   296]
 ...
 [  101  2054  2820 ...     0     4    58]
 [  101  2054  2003 ...     0   466   516]
 [  101  1999  2054 ...     0   303   307]], shape=(32, 945), dtype=int32)


InvalidArgumentError: ignored

In [ ]:
max(sentences_lengths)